<a href="https://colab.research.google.com/github/shweetak/Sweta/blob/main/Order_Data_analysis_spark_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install Apache Spark 3.0.1 with Hadoop 2.7 from here.
!wget https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz

# Now, we just need to unzip that folder.
!tar -xvzf spark-3.0.0-bin-hadoop2.7.tgz
!pip install findspark


import os
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"
import findspark
findspark.init()


In [10]:
# 1. Problem Statement: What is the daily product revenue for CLOSED or COMPLETE orders

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

status_filter = df["_c3"].isin(["CLOSED", "COMPLETE"])
filtered_df = df.filter(status_filter)

revenue_df = filtered_df.groupBy(date_format("_c1", "yyyy-MM-dd").alias("date")).agg(sum("_c2").alias("daily_product_revenue"))

revenue_df.show(truncate=False)

+----------+---------------------+
|date      |daily_product_revenue|
+----------+---------------------+
|2014-02-16|757942.0             |
|2014-02-22|365090.0             |
|2014-05-27|240185.0             |
|2014-07-14|453331.0             |
|2013-08-20|535789.0             |
|2013-12-28|662309.0             |
|2013-11-08|450352.0             |
|2014-03-17|259678.0             |
|2014-03-20|517343.0             |
|2013-09-27|815868.0             |
|2014-07-24|536326.0             |
|2014-02-08|445823.0             |
|2014-05-23|445204.0             |
|2013-11-05|799736.0             |
|2013-10-05|552018.0             |
|2013-10-29|312465.0             |
|2014-05-01|379722.0             |
|2014-06-02|582818.0             |
|2014-04-02|616979.0             |
|2014-07-11|334643.0             |
+----------+---------------------+
only showing top 20 rows



In [12]:
#2. Load the required data in to DF like categories, customer,departments,order_items,orders and products

from pyspark.sql import SparkSession
from pyspark.sql.types import *
spark = SparkSession.builder.master("local").appName("Order_data_Analysis").getOrCreate()

categories = spark.read.csv(path = "/categories",header=False)
customer = spark.read.csv(path = "/customers",header=False)
departments = spark.read.csv(path = "/departments",header=False)
order_items = spark.read.csv(path = "/order_items",header=False)
orders = spark.read.csv(path = "/orders",header=False)
products = spark.read.csv(path = "/products",header=False)
categories.show(n=5)
customer.show(n=5)
departments.show(n=5)
order_items.show(n=5)
orders.show(n=5)
products.show(n=5)



+---+---+-------------------+
|_c0|_c1|                _c2|
+---+---+-------------------+
|  1|  2|           Football|
|  2|  2|             Soccer|
|  3|  2|Baseball & Softball|
|  4|  2|         Basketball|
|  5|  2|           Lacrosse|
+---+---+-------------------+
only showing top 5 rows

+---+-------+---------+---------+---------+--------------------+-----------+---+-----+
|_c0|    _c1|      _c2|      _c3|      _c4|                 _c5|        _c6|_c7|  _c8|
+---+-------+---------+---------+---------+--------------------+-----------+---+-----+
|  1|Richard|Hernandez|XXXXXXXXX|XXXXXXXXX|  6303 Heather Plaza|Brownsville| TX|78521|
|  2|   Mary|  Barrett|XXXXXXXXX|XXXXXXXXX|9526 Noble Embers...|  Littleton| CO|80126|
|  3|    Ann|    Smith|XXXXXXXXX|XXXXXXXXX|3422 Blue Pioneer...|     Caguas| PR|00725|
|  4|   Mary|    Jones|XXXXXXXXX|XXXXXXXXX|  8324 Little Common| San Marcos| CA|92069|
|  5| Robert|   Hudson|XXXXXXXXX|XXXXXXXXX|10 Crystal River ...|     Caguas| PR|00725|
+---+----

In [15]:
# 3. Get the count for each order status

order_status_count = (orders.groupBy("_c3").agg(count("_c0").alias("order_status_count")).withColumnRenamed("_c3", "order_status"))

order_status_count.show(truncate=False)

+---------------+------------------+
|order_status   |order_status_count|
+---------------+------------------+
|PENDING_PAYMENT|15030             |
|COMPLETE       |22899             |
|ON_HOLD        |3798              |
|PAYMENT_REVIEW |729               |
|PROCESSING     |8275              |
|CLOSED         |7556              |
|SUSPECTED_FRAUD|1558              |
|PENDING        |7610              |
|CANCELED       |1428              |
+---------------+------------------+



In [18]:
# 4. Filter only COMPLETE or CLOSED orders

filtered_df = df.filter(df["_c3"].isin(["COMPLETE", "CLOSED"]))
filtered_df.show()

+---+--------------------+-----+--------+
|_c0|                 _c1|  _c2|     _c3|
+---+--------------------+-----+--------+
|  1|2013-07-25 00:00:...|11599|  CLOSED|
|  3|2013-07-25 00:00:...|12111|COMPLETE|
|  4|2013-07-25 00:00:...| 8827|  CLOSED|
|  5|2013-07-25 00:00:...|11318|COMPLETE|
|  6|2013-07-25 00:00:...| 7130|COMPLETE|
|  7|2013-07-25 00:00:...| 4530|COMPLETE|
| 12|2013-07-25 00:00:...| 1837|  CLOSED|
| 15|2013-07-25 00:00:...| 2568|COMPLETE|
| 17|2013-07-25 00:00:...| 2667|COMPLETE|
| 18|2013-07-25 00:00:...| 1205|  CLOSED|
| 22|2013-07-25 00:00:...|  333|COMPLETE|
| 24|2013-07-25 00:00:...|11441|  CLOSED|
| 25|2013-07-25 00:00:...| 9503|  CLOSED|
| 26|2013-07-25 00:00:...| 7562|COMPLETE|
| 28|2013-07-25 00:00:...|  656|COMPLETE|
| 32|2013-07-25 00:00:...| 3960|COMPLETE|
| 35|2013-07-25 00:00:...| 4840|COMPLETE|
| 37|2013-07-25 00:00:...| 5863|  CLOSED|
| 45|2013-07-25 00:00:...| 2636|COMPLETE|
| 51|2013-07-25 00:00:...|12271|  CLOSED|
+---+--------------------+-----+--

In [22]:
# 5. join the products , order_items and orders tables and calculate daily product revenue

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.master("local").appName("DailyProductRevenue").getOrCreate()

joined_df = (order_items.alias("oi").join(products.alias("p"), col("oi._c2") == col("p._c0"), "inner").join(orders.alias("o"), col("oi._c1") == col("o._c0"), "inner"))

daily_product_revenue = (joined_df.withColumn("order_date", to_date("o._c1")).groupBy("order_date", "p._c4") .agg(sum("oi._c4").alias("daily_revenue")).orderBy("order_date"))
daily_product_revenue.show(truncate=False)


+----------+------+------------------+
|order_date|_c4   |daily_revenue     |
+----------+------+------------------+
|2013-07-25|149.99|299.98            |
|2013-07-25|199.99|7399.629999999995 |
|2013-07-25|599.99|599.99            |
|2013-07-25|51.99 |207.96            |
|2013-07-25|79.99 |559.93            |
|2013-07-25|31.99 |543.8299999999999 |
|2013-07-25|15.99 |159.9             |
|2013-07-25|9.99  |19.98             |
|2013-07-25|109.99|219.98            |
|2013-07-25|50.0  |5100.0            |
|2013-07-25|108.0 |216.0             |
|2013-07-25|21.99 |21.99             |
|2013-07-25|19.99 |79.96             |
|2013-07-25|299.98|9599.359999999993 |
|2013-07-25|99.0  |297.0             |
|2013-07-25|99.99 |8499.149999999998 |
|2013-07-25|129.99|5589.569999999994 |
|2013-07-25|25.0  |100.0             |
|2013-07-25|34.99 |69.98             |
|2013-07-25|399.98|10799.459999999994|
+----------+------+------------------+
only showing top 20 rows



In [ ]:
# 6. Write the data in to the table Daily product revenue in Hive

orders = "daily_revenue_sweta.daily_product_revenue"

daily_product_revenue.write.mode("overwrite").saveAsTable(orders)